In [1]:
%load_ext autoreload
%autoreload 2

In [96]:
from openpharmacophore.dynophore import Dynophore
import os
import mdtraj as md
import numpy as np
from collections import namedtuple
from tqdm.auto import tqdm

import pandas as pd

# Pharmacophoric points analysis for MD simulation of ERalpha

### Load trajectory

In [3]:
traj_file = "../../data/trajectories/ERalpha.h5"
traj = md.load(traj_file)
print(traj)
print(traj.topology)

<mdtraj.Trajectory with 1000 frames, 27425 atoms, 8028 residues, and unitcells>
<mdtraj.Topology with 4 chains, 8028 residues, 27425 atoms, 19689 bonds>


## Get pharmacophores from the first and last frame only

In [4]:
dynophore = Dynophore(traj, method="last")
dynophore.get_pharmacophores()
dynophore.pharmacophores

[StructuredBasedPharmacophore(n_elements: 9),
 StructuredBasedPharmacophore(n_elements: 6)]

In [5]:
dynophore.pharmacophores[0].elements

[HbAcceptorSphereAndVector(center: (28.34, 17.742, 25.083); radius: 1.0; direction: (0.4886, -0.87, -0.066)),
 HbAcceptorSphereAndVector(center: (37.768, 12.081, 23.029); radius: 1.0; direction: (-0.6034, 0.2741, 0.7489)),
 HbDonorSphereAndVector(center: (28.34, 17.742, 25.083); radius: 1.0; direction: (0.1793, 0.4914, 0.8523)),
 HbDonorSphereAndVector(center: (37.768, 12.081, 23.029); radius: 1.0; direction: (-0.3717, -0.9283, 0.0118)),
 HydrophobicitySphere(center: (33.452, 15.524, 23.64); radius: 1.0),
 HydrophobicitySphere(center: (31.2353, 16.7123, 24.6887); radius: 1.0),
 HydrophobicitySphere(center: (30.49, 15.3043, 23.3997); radius: 1.0),
 HydrophobicitySphere(center: (35.225, 12.201, 24.116); radius: 1.0),
 HydrophobicitySphere(center: (36.287, 12.367, 21.152); radius: 1.0)]

In [6]:
dynophore.pharmacophores[0].show()

NGLWidget()

In [7]:
dynophore.pharmacophores[-1].elements

[HbAcceptorSphereAndVector(center: (21.4, 23.433, 23.959); radius: 1.0; direction: (0.9029, 0.0437, 0.4276)),
 HbAcceptorSphereAndVector(center: (32.187, 20.784, 24.975); radius: 1.0; direction: (-0.8643, 0.3032, 0.4012)),
 HbDonorSphereAndVector(center: (21.4, 23.433, 23.959); radius: 1.0; direction: (-0.9257, -0.2297, 0.3006)),
 HydrophobicitySphere(center: (24.896, 20.102, 23.084); radius: 1.0),
 HydrophobicitySphere(center: (30.969, 19.259, 23.494); radius: 1.0),
 HydrophobicitySphere(center: (25.44, 22.574, 23.702); radius: 1.0)]

In [8]:
dynophore.pharmacophores[-1].show()

NGLWidget()

## Pharmacophore Analysis

### Get Pharmacophores for each frame of the trajectory

In [9]:
import time

start_time = time.time()

In [11]:
dynophore = Dynophore(traj)
dynophore._all_frames_pharmacophores()
print("--- %s seconds ---" % (time.time() - start_time))

--- 3077.8478903770447 seconds ---


In [13]:
print(f"Number of pharmacophores {len(dynophore.pharmacophores)}")
print(dynophore.pharmacophores[0:5])

Number of pharmacophores 1000
[StructuredBasedPharmacophore(n_elements: 9), StructuredBasedPharmacophore(n_elements: 6), StructuredBasedPharmacophore(n_elements: 7), StructuredBasedPharmacophore(n_elements: 8), StructuredBasedPharmacophore(n_elements: 7)]


### Get all unique pharmacophoric features

In [74]:
class PharmacophoreFeature():
    
    def __init__(self, name, indices, count=0):
        self.name = name
        self.indices = indices
        self.count = 0

    def __eq__(self, other):
        if self.name == other.name and self.indices == other.indices:
            return True
        return False
        
        
    def __repr__(self):
        return f"Feature(name={self.name}, indices={self.indices}, count={self.count})"



In [75]:
all_features = []
for pharma in tqdm(dynophore.pharmacophores):
    for p in pharma.elements:
        f = PharmacophoreFeature(p.short_name, p.atoms_inxs)
        all_features.append(f)
        
print(len(all_features))

  0%|          | 0/1000 [00:00<?, ?it/s]

5995


In [77]:
unique_feats = []
for feat in all_features:
    
    is_unique = True
    for unique_f in unique_feats:
        if feat == unique_f:
            unique_f.count += 1
            is_unique = False
            break
    if is_unique:
        unique_feats.append(feat)
    
print(len(unique_feats))

19


In [104]:
unique_feats[0:3]

[Feature(name=A, indices={4}, count=878),
 Feature(name=A, indices={19}, count=990),
 Feature(name=D, indices={4}, count=810)]

In [107]:
feature_mapper = {
            "hb acceptor": "A",
            "hb donor": "D",
            "aromatic ring": "R",
            "hydrophobicity": "H",
            "positive charge": "P",
            "negative charge": "N",
            "excluded sphere": "E",
            "included sphere": "I"
        }

inverse_feature_mapper = {v:k for k, v in feature_mapper.items()}

names = []
feat_type_list = []
percentage = []
indices = []
n_timesteps = len(dynophore.pharmacophores)
for feat in unique_feats:
    feat_type = inverse_feature_mapper[feat.name] + " "
    feat_num = 1
    full_name = feat_type + str(feat_num)
    if full_name not in names:
        names.append(full_name)
    else:
        while True:
            feat_num += 1
            full_name = feat_type + str(feat_num)
            if full_name not in names:
                names.append(full_name)
                break
    
    percent = (feat.count / n_timesteps) * 100
    
    feat_type_list.append(feat.name)
    percentage.append(percent)
    indices.append(feat.indices)


In [108]:
frequency = pd.DataFrame().from_dict({
    "Feature Name": names,
    "Type": feat_type_list,
    "Frequency(%)": percentage,
    "Atoms Indices": indices
})

frequency

,Feature Name,Type,Frequency(%),Atoms Indices
0,hb acceptor 1,A,87.8,{4}
1,hb acceptor 2,A,99.0,{19}
2,hb donor 1,D,81.0,{4}
3,hb donor 2,D,56.0,{19}
4,hydrophobicity 1,H,0.4,{10}
5,hydrophobicity 2,H,48.7,{1}
6,hydrophobicity 3,H,17.9,{7}
7,hydrophobicity 4,H,26.8,{20}
8,hydrophobicity 5,H,32.2,{17}
9,hydrophobicity 6,H,59.8,{5}


## Pharmacophore model from most frequent features

In [109]:
threshold = 50
model_feats = frequency[frequency["Frequency(%)"] >= threshold]
model_feats

,Feature Name,Type,Frequency(%),Atoms Indices
0,hb acceptor 1,A,87.8,{4}
1,hb acceptor 2,A,99.0,{19}
2,hb donor 1,D,81.0,{4}
3,hb donor 2,D,56.0,{19}
9,hydrophobicity 6,H,59.8,{5}
